In [ ]:
from pop import Util, Pilot
from pop.Util import imshow
import numpy as np
import cv2
import math
import time
import threading
import sys
# import queue
from scipy.ndimage import rotate
import multiprocessing as mp
import re

import subprocess as sp, time
from IPython.display import display, Javascript
from ipywidgets import widgets

import subprocess as sp, time

img_size_x = 600
img_size_y = 600

ICP_MAP_SENTOR_X = img_size_x // 2
ICP_MAP_SENTOR_Y = img_size_y // 2

img_exp_x = 900
img_exp_y = 900

exp_MAP_SENTOR_X = img_exp_x // 2
exp_MAP_SENTOR_Y = img_exp_y // 2


indexColor = 0
baseIMGSIZE_x = 2200
baseIMGSIZE_y = 2200
baseIMGsentor_x, baseIMGsentor_y = int(baseIMGSIZE_x // 2) , int(baseIMGSIZE_y // 2)
before_point_x, before_point_y = baseIMGsentor_x, baseIMGsentor_y

bot = Pilot.SerBot()
Util.enable_imshow()

In [ ]:
# colors
steelblue = (180,130,70)
green = (0,255,108)
white = (255,255,255)
azure = (255,255,240)
aqua = (255,255,0)
red = (0,0,255)
black = (0,0,0)
yellow = (0,255,255)
orange = (0, 165, 255)
gray = (128,128,128)

rainbow = [(0,0,255), (0, 140, 255), (0,255,255), (0,128,0), (255,0,0), (130,0,75), (128,0,128), (255,255,255)]

<h3> ICP Algorithm (Iterative Closest Point) <h3/>

In [ ]:
def ICP(model_x, model_y, sample_x, sample_y):
    dis_tuple = np.zeros((len(model_x), 3), dtype=np.uint16)
    dis_sum = 0
    count = 0
    for i in range(len(model_x)):
        base_x = int(model_x[i] - sample_x[0])
        base_y = int(model_y[i] - sample_y[0])
        baseDis = math.pow(base_x, 2) + math.pow(base_y, 2)
        baseDisMin = math.sqrt(baseDis)
        for j in range(len(sample_x)):
            move_x = int(model_x[i] - sample_x[j])
            move_y = int(model_y[i] - sample_y[j])
            moveDisPOW = math.pow(move_x, 2) + math.pow(move_y, 2)
            moveDis = math.sqrt(moveDisPOW)
            if moveDis <= baseDisMin:
                baseDisMin = moveDis
                dis_tuple[i][0] = int(sample_x[j])
                dis_tuple[i][1] = int(sample_y[j])
                dis_tuple[i][2] = moveDis
                dis_sum += moveDis
                count += 1
    
    dis_avr = round(dis_sum / count, 6)
    for i in range(len(dis_tuple)):
        dis_t_p = math.pow(dis_tuple[i][2], 2)
        dis_a_p = math.pow(dis_avr, 2)
        if dis_t_p > dis_a_p:
            dis_tuple[i][0] = int(model_x[i])
            dis_tuple[i][1] = int(model_y[i])
            dis_tuple[i][2] = 0
    return dis_tuple

In [ ]:
def size_change(sample_x, sample_y):
    global img_exp_x, img_exp_y, exp_MAP_SENTOR_X, exp_MAP_SENTOR_Y, ICP_MAP_SENTOR_X, ICP_MAP_SENTOR_Y
    sample = np.zeros((img_exp_x, img_exp_y), dtype=np.uint16)
    for i in range(len(sample_x)):
        try:
            temp_x = int(exp_MAP_SENTOR_X - (ICP_MAP_SENTOR_X - sample_x[i]))
            temp_y = int(exp_MAP_SENTOR_Y - (ICP_MAP_SENTOR_Y - sample_y[i]))
            sample[temp_x][temp_y] = 1
        except IndexError:
            print(int(round(exp_MAP_SENTOR_X - (ICP_MAP_SENTOR_X - sample_x[i]))), int(round(exp_MAP_SENTOR_Y - (ICP_MAP_SENTOR_Y - sample_y[i]))))
            print(len(sample_x), len(sample_y))
            pass
    return sample

In [ ]:
def photo(dis_tuple, model_x, model_y): #ch_point_x관련해서 고칠 필요 있음
    global indexColor, rainbow, ICPMap, ICP_MAP_SENTOR_X, ICP_MAP_SENTOR_Y, exp_MAP_SENTOR_X, exp_MAP_SENTOR_Y, img_exp_x, img_exp_y
    global ICPMap
    cv2.imwrite('BaseArray.jpg', ICPMap)
    baseMapIMG = cv2.imread('BaseArray.jpg', cv2.IMREAD_COLOR)
    
    for i in range(len(model_x)):
        temp_x = int(model_x[i])
        temp_y = int(model_y[i])
        baseMapIMG = cv2.circle(baseMapIMG, (temp_y, temp_x), 4, tuple(rainbow[1]), -1)
        
    for i in range(len(dis_tuple)):
        temp_x = int(model_x[i])
        temp_y = int(model_y[i])
        temp_dis_x = int(dis_tuple[i][0])
        temp_dis_y = int(dis_tuple[i][1])
        baseMapIMG = cv2.circle(baseMapIMG, (temp_dis_y, temp_dis_x), 4, tuple(rainbow[3]), -1)
        baseMapIMG = cv2.line(baseMapIMG, (temp_y, temp_x), (temp_dis_y, temp_dis_x),tuple(rainbow[0]), 1)
            
    baseMapIMG = cv2.circle(baseMapIMG, (exp_MAP_SENTOR_Y, exp_MAP_SENTOR_X), 10, tuple(rainbow[3]), -1)
    baseMapIMG = cv2.circle(baseMapIMG, (exp_MAP_SENTOR_Y, exp_MAP_SENTOR_X), 10, tuple(rainbow[1]), -1)
    
    cv2.imwrite('BaseArray.jpg', baseMapIMG)
    cv2.imshow('BaseArray.jpg', baseMapIMG, width=600, height=600)

In [ ]:
img_count = 1
img_path = "/home/soda/Project/python/notebook/KJJ/project/splitprocessing/MAP/"
def photo_map(sample_x, sample_y, now_point_x, now_point_y, trace):
    global ICP_MAP_SENTOR_X, ICP_MAP_SENTOR_Y
    global exp_MAP_SENTOR_X, exp_MAP_SENTOR_Y
    global indexColor, rainbow, color_division, white, aqua, gray
    global img_count, MapIMG
    
    if img_count < 10:
        MapIMG = cv2.imread(img_path + 'MapArray' + "00" + str(img_count-1) + '.jpg', cv2.IMREAD_COLOR)
    elif img_count < 100:
        if img_count == 10:
            MapIMG = cv2.imread(img_path + 'MapArray' + "00" + str(img_count-1) + '.jpg', cv2.IMREAD_COLOR)
        else :
            MapIMG = cv2.imread(img_path + 'MapArray' + "0" + str(img_count-1) + '.jpg', cv2.IMREAD_COLOR)
    else :
        if img_count == 100:
            MapIMG = cv2.imread(img_path + 'MapArray' + "0" + str(img_count-1) + '.jpg', cv2.IMREAD_COLOR)
        else :
            MapIMG = cv2.imread(img_path + 'MapArray' + str(img_count-1) + '.jpg', cv2.IMREAD_COLOR)
#     MapIMG = cv2.imread('MapArray.jpg', cv2.IMREAD_COLOR)
    color = 7
    count = 0
    while count < 2:
        for i in range(len(sample_x)):
            temp_x = exp_MAP_SENTOR_X - sample_x[i]
            temp_y = exp_MAP_SENTOR_Y - sample_y[i]
            step = distance(temp_x, temp_y)
            
            now_x = int(now_point_x - temp_x)
            now_y = int(now_point_y - temp_y)
            if step < round(ICP_MAP_SENTOR_X * 0.7):
                if count == 0:
                    MapIMG = cv2.line(MapIMG, (now_point_y, now_point_x), (now_y, now_x), tuple(gray), 5)
                else :
                    MapIMG = cv2.circle(MapIMG, (now_y, now_x), 3, tuple(rainbow[color]), -1)
        count += 1

    for i in range(len(trace)):
        MapIMG = cv2.circle(MapIMG, tuple(trace[i]), 25, tuple(white), -1)
        k = i - 1
        MapIMG = cv2.line(MapIMG, tuple(trace[i]), tuple(trace[k]), tuple(aqua), 3)
    
#         if step <= round((img_exp_x * 0.15), 3) :
#             color = 0
#         elif step <= round((img_exp_x * 0.3), 3) :
#             color = 1
#         elif step <= round((img_exp_x * 0.45), 3) :
#             color = 2
#         elif step <= round((img_exp_x * 0.6), 3) :
#             color = 3
#         elif step <= round((img_exp_x * 0.75), 3) :
#             color = 4
#         elif step <= round((img_exp_x * 0.9), 3) :
#             color = 5
#         else :
#             color = 6
    
    if img_count < 10:
        cv2.imwrite(img_path + 'MapArray' + "00" + str(img_count) + '.jpg', MapIMG)
        cv2.imshow(img_path + 'MapArray' + "00" + str(img_count) + '.jpg', MapIMG, width=600, height=600)
    elif img_count < 100:
        cv2.imwrite(img_path + 'MapArray' + "0" + str(img_count) + '.jpg', MapIMG)
        cv2.imshow(img_path + 'MapArray' + "0" + str(img_count) + '.jpg', MapIMG, width=600, height=600)
    else :
        cv2.imwrite(img_path + 'MapArray' + str(img_count) + '.jpg', MapIMG)
        cv2.imshow(img_path + 'MapArray' + str(img_count) + '.jpg', MapIMG, width=600, height=600)
    img_count = img_count + 1
#     cv2.imwrite('MapArray.jpg', MapIMG)
#     cv2.imshow('MapArray.jpg', MapIMG, width=600, height=600)
    
def distance(x, y):
    r = math.sqrt(math.pow(x, 2) + math.pow(y, 2))
    return r

In [ ]:
def move(model_x, model_y, sample_x, sample_y, before_dis, before_dif_x, before_dif_y, move_point_x, move_point_y):
    # test1.ipynb 참조
    global ICP_MAP_SENTOR_X, ICP_MAP_SENTOR_Y
    dif_x, dif_y = before_dif_x, before_dif_y
    avr_x = avr_y = avr_dis = 0
    
    dis_tuple = ICP(model_x, model_y, sample_x, sample_y)
    avr_x, avr_y, avr_dis = dis_diff(model_x, model_y, dis_tuple, before_dif_x, before_dif_y)
    dif_x, sample_x, move_point_x = move_x(avr_x, dif_x, sample_x, move_point_x, avr_dis)
    
    dis_tuple = ICP(model_x, model_y, sample_x, sample_y)
    avr_x, avr_y, avr_dis = dis_diff(model_x, model_y, dis_tuple, dif_x, before_dif_y)
    dif_y, sample_y, move_point_y = move_y(avr_y, dif_y, sample_y, move_point_y, avr_dis)
    
    if before_dif_x == dif_x and before_dif_y == dif_y and before_dis == avr_dis:
        return False, sample_x, sample_y, avr_dis, dif_x, dif_y, move_point_x, move_point_y
    else :
        return True, sample_x, sample_y, avr_dis, dif_x, dif_y, move_point_x, move_point_y
    
def dis_diff(model_x, model_y, dis_tuple, before_dif_x, before_dif_y):
    stock_x = 0
    stock_y = 0
    count = 0
    dis_sum = 0
    for i in range(len(dis_tuple)):
        if dis_tuple[i][2] != 0:
            temp_a = (model_x[i] - dis_tuple[i][0])
            temp_b = (model_y[i] - dis_tuple[i][1])
    #             if math.fabs(temp_a) <= math.fabs(before_dif_x) or math.fabs(temp_b) <= math.fabs(before_dif_y):
            stock_x += temp_a
            stock_y += temp_b
            dis = round(math.sqrt(math.pow(temp_a, 2) + math.pow(temp_b, 2)), 6)
            dis_sum += dis
            count += 1
    
    if stock_x != 0:
        avr_x = round(stock_x / count, 6)
    else :
        avr_x = 0
    if stock_y != 0:
        avr_y = round(stock_y / count, 6)
    else :
        avr_y = 0
    if dis_sum != 0:
        avr_dis = round(dis_sum / count, 6)
    else :
        avr_dis = 0
    return avr_x, avr_y, avr_dis
def move_x(avr_x, dif_x, sample_x, move_point_x, avr_dis):
    if (avr_x < math.fabs(dif_x) and avr_x > 0) or (avr_x > -math.fabs(dif_x) and avr_x < 0):
        dif_x = round(avr_x*1.5, 6)
        sample_x = sample_x + round(avr_x)
        move_point_x += round(avr_x, 6)
    else :
        dif_x = round(avr_x*avr_dis*1.5, 6)
        sample_x = sample_x + round(avr_x*avr_dis)
        move_point_x += round(avr_x*avr_dis, 6)
        pass
    return dif_x, sample_x, move_point_x
def move_y(avr_y, dif_y, sample_y, move_point_y, avr_dis):
    if (avr_y < math.fabs(dif_y) and avr_y > 0) or (avr_y > -math.fabs(dif_y) and avr_y < 0):
        dif_y = round(avr_y*1.5, 6)
        sample_y = sample_y + round(avr_y)
        move_point_y += round(avr_y, 6)
    else :
        dif_y = round(avr_y*avr_dis*1.5, 6)
        sample_y = sample_y + round(avr_y*avr_dis)
        move_point_y += round(avr_y*avr_dis, 6)
        pass
    return dif_y, sample_y, move_point_y

In [ ]:
# 순서가 정해져 있는 각 점마다 회전한 각도의 평큔을 구함.
# 첫 번째는 ICP한 점을 기준으로 만들고
# 두 번째는 model과 sample과의 점을 기준으로 만듬.
# test11 참조

angle_temp_sign_P = 360
angle_temp_sign_M = -360

def rotate_init():
    global angle_temp_sign_P, angle_temp_sign_M
    angle_temp_sign_P = 360
    angle_temp_sign_M = -360

def my_rotate(model_x, model_y, sample_x, sample_y, angle_sum, before_angle):
    global ICP_MAP_SENTOR_X, ICP_MAP_SENTOR_Y, angle_temp_sign_P, angle_temp_sign_M
    global img_exp_x, img_exp_y, exp_MAP_SENTOR_X, exp_MAP_SENTOR_Y
    status = True
    angle_sign = -1
    angle_sum = angle_sum
    sumangle = count = 0
    dis_tuple = ICP(model_x, model_y, sample_x, sample_y)
    
    for i in range(len(dis_tuple)):
        if dis_tuple[i][2] != 0:
            point_x = exp_MAP_SENTOR_X - model_x[i]
            point_y = exp_MAP_SENTOR_Y - model_y[i]
            r_model = round(math.sqrt(math.pow(point_x, 2) + math.pow(point_y, 2)), 6)
            # 각 x, y 포인트의 부호에 따라 계산된 값의 최종값의 부호를 결정.

            r_x = exp_MAP_SENTOR_X - dis_tuple[i][0]
            r_y = exp_MAP_SENTOR_Y - dis_tuple[i][1]
            r_sample = round(math.sqrt(math.pow(r_x, 2) +  math.pow(r_y, 2)), 6)

            angle_som = 0

            try:
#                 cosA = (b^2 + c^2 - a^2)/(2bc)
#                 A = arccos((b^2 + c^2 - a^2)/(2bc))
                cos_som = round((math.pow(r_model, 2) + math.pow(r_sample, 2) - math.pow(dis_tuple[i][2], 2)) / (2 * r_model * r_sample), 6)
                angle_som = round(math.degrees(math.acos(cos_som)), 6)
                sumangle = round(sumangle + (angle_som), 6)
                count = count + 1

            except ValueError:
                sumangle = round(sumangle + (angle_som), 6)
                count = count + 1
                pass
                
    if sumangle != 0:
        angle_temp = round(sumangle / count, 6)
    else :
        angle_temp = 0
        
    if before_angle == 0:
        before_sign = 1
    else :
        before_sign = (before_angle / math.fabs(before_angle))

    if angle_temp != 0:
        if math.fabs(angle_temp) < math.fabs(before_angle) and angle_temp_sign_P > math.fabs(angle_temp) and angle_temp_sign_M < math.fabs(angle_temp):
            angle_temp = round(angle_temp * before_sign, 6)
            if before_sign > 0:
                angle_temp_sign_P = angle_temp
            elif before_sign < 0:
                angle_temp_sign_M = angle_temp
                
        elif angle_temp >= math.fabs(before_angle):
            if before_sign < 0 and angle_temp_sign_P > angle_temp:
                angle_temp_sign_P = round(angle_temp, 6)
            elif before_sign > 0 and math.fabs(angle_temp_sign_M) > angle_temp:
                angle_temp_sign_M = round(angle_temp * angle_sign, 6)
                angle_temp = angle_temp_sign_M
            else : 
                angle_temp = (angle_temp_sign_P + angle_temp_sign_M)
                status = False
                
        else : 
            angle_temp = (angle_temp_sign_P + angle_temp_sign_M)
            status = False
            
    else :
        pass
    
    angle = round(angle_temp/2, 6)
    angle_sum += angle
    
    angle_temp = round(angle_temp, 6)
    angle_sum = round(angle_sum, 6)
    angle_ref = int(angle)
    
    sample = np.zeros((img_exp_x, img_exp_y), dtype=np.uint16)
    for i in range(len(sample_x)):
        temp_x = int(sample_x[i])
        temp_y = int(sample_y[i])
        sample[temp_x][temp_y] = 1
    sample = rotate(sample, angle=-angle_ref, reshape=False)
    sample_x, sample_y = np.where(sample==1)
    
    print(angle_sum)
    lab4[-1].value="sum, temp: {} [']".format((angle_sum, angle_temp))
    return status, sample_x, sample_y, angle_sum, angle_temp

In [ ]:
def optimize(model_x, model_y, sample_x, sample_y, move_point_x, move_point_y):
    # False이면 값이 줄지 않는 것. True이면 값이 줄어들거나 동작시킬 필요가 있는 것.
    #이것을 조건에 맞을 때까지 반복한다.
    move_status = False
    rotate_status = True
    
    # 각도 회전을 위한 초기화
    angle = 0
    before_angle = 0
    angle_sum = 0
    angle_sum_o = angle_sum_i = 0
    rotate_init()

    dif_x = dif_y = before_dif_x = before_dif_y = before_dis = 200
    dif_dis = 0
    
    dis_tuple = ICP(model_x, model_y, sample_x, sample_y)
    photo(dis_tuple, model_x, model_y)
    
    x = y = a = 0
    x_d = y_d = a_d = 1
    
    while move_status or rotate_status :
        # 각 if문에 조건에 걸리면 회전 및 이동 시켜라.
        if move_status == True: # move 함수가 작동됨에 따라 값이 증가 하면 
            move_status, sample_x, sample_y, dif_dis, dif_x, dif_y, move_point_x, move_point_y = move(model_x, model_y, sample_x, sample_y, 
                                                                                                 before_dis, before_dif_x, before_dif_y, move_point_x, move_point_y)
            x_d = dif_x
            y_d = dif_y
            if move_status == False:
                rotate_status = True
                dif_x = dif_y = before_dif_x = before_dif_y = before_dis = 200
            elif move_status == True:
                rotate_status = False
                before_dis = dif_dis
                before_dif_x, before_dif_y = dif_x, dif_y
    
        if rotate_status == True:
            rotate_status, sample_x, sample_y, angle_sum_o, angle = my_rotate(model_x, model_y, sample_x, sample_y, angle_sum_i, before_angle)
            a_d = angle
            angle_sum_i = angle_sum_o
#             angle_sum += angle_sum_o
            if rotate_status == False:
                move_status = True
                rotate_init()
                angle = before_angle = 0
            elif rotate_status == True:
                move_status = False
                before_angle = angle
        
        dis_tuple = ICP(model_x, model_y, sample_x, sample_y)
        photo(dis_tuple, model_x, model_y)
        
        if x == x_d and y == y_d and a == a_d:
            print("end")
            break
        else :
            print("again", end=", ")
            x = x_d
            y = y_d
            a = a_d
    
    move_point_x = round(move_point_x, 6)
    move_point_y = round(move_point_y, 6)
    angle_sum = angle_sum_o
    angle_sum = round(angle_sum, 6)
    
    lab1[-1].value="mx, my, as: {} [']".format((move_point_x, move_point_y, angle_sum))
    return angle_sum, move_point_x, move_point_y

In [ ]:
ICPMap = np.zeros((img_exp_x, img_exp_y), dtype=np.uint16)
cv2.imwrite('BaseArray.jpg', ICPMap)
cv2.imshow('BaseArray.jpg', ICPMap, width=600, height=600)

MapIMG = np.zeros((baseIMGSIZE_x, baseIMGSIZE_y), dtype=np.uint16)
# cv2.imwrite('MapArray.jpg', Map)
# cv2.imshow('MapArray.jpg', Map, width=600, height=600)
cv2.imwrite(img_path + 'MapArray' + "00" + str(img_count-1) + '.jpg', MapIMG)
cv2.imshow(img_path + 'MapArray' + "00" + str(img_count-1) + '.jpg', MapIMG, width=600, height=600)
cv2.destroyAllWindows()

filecount = 16
with open("/home/soda/Project/python/notebook/KJJ/project/splitprocessing/point.txt", "r") as f:
    data = f.readlines()
    pass
d_x = data[filecount]
d_y = data[filecount+1]
sd_x = re.findall("\d+", d_x)
sd_y = re.findall("\d+", d_y)
ind_x = list(map(int, sd_x))
ind_y = list(map(int, sd_y))
model_x = np.array(ind_x)
model_y = np.array(ind_y)

model = size_change(model_x, model_y)
model_x, model_y = np.where(model==1)

filecount += 4
d_x = data[filecount]
d_y = data[filecount+1]
sd_x = re.findall("\d+", d_x)
sd_y = re.findall("\d+", d_y)
ind_x = list(map(int, sd_x))
ind_y = list(map(int, sd_y))
sample_x = np.array(ind_x)
sample_y = np.array(ind_y)

sample = size_change(sample_x, sample_y)
sample_x, sample_y = np.where(sample==1)

lab1 = []
lab1.append(widgets.Label(value="a: 0 [']"))
display(lab1[-1])
lab2 = []
lab2.append(widgets.Label(value="b: 0 [']"))
display(lab2[-1])
lab3 = []
lab3.append(widgets.Label(value="c: 0 [']"))
display(lab3[-1])
lab4 = []
lab4.append(widgets.Label(value="d: 0 [']"))
display(lab4[-1])


move_point_x = 0
move_point_y = 0

ref_point_x = h_p_x = baseIMGsentor_x
ref_point_y = h_p_y = baseIMGsentor_y

ref = math.sqrt(math.pow(ref_point_x, 2) + math.pow(ref_point_y, 2))
h_p = math.sqrt(math.pow(h_p_x, 2) + math.pow(h_p_y, 2))

angle_f = ref_angle = 0

trace = []
trace.append(tuple((ref_point_y, ref_point_x)))

dis_tuple = ICP(model_x, model_y, sample_x, sample_y)
photo_map(sample_x, sample_y, h_p_x, h_p_y, trace)

serbot_angle = -360

while True:
    # 좌표 교환
    model_x = sample_x
    model_y = sample_y
    
    print("!", filecount,"!", end=", ")
    filecount += 4
    d_x = data[filecount]
    d_y = data[filecount+1]
    sd_x = re.findall("\d+", d_x)
    sd_y = re.findall("\d+", d_y)
    ind_x = list(map(int, sd_x))
    ind_y = list(map(int, sd_y))
    sample_x = np.array(ind_x)
    sample_y = np.array(ind_y)
    pass

    sample = size_change(sample_x, sample_y)
    sample_x, sample_y = np.where(sample==1)
    
    angle_now, move_point_x, move_point_y = optimize(model_x, model_y, sample_x, sample_y, move_point_x, move_point_y)
    angle_f = round(angle_f + angle_now, 6)
    ref_angle = round(angle_f)
    
    move_r = round(distance(move_point_x, move_point_y), 6)
    tan = round(math.atan2(move_point_y, move_point_x), 6)
    tan_degree = round(math.degrees(tan), 6)

    x_x = round(move_r * math.cos(math.radians(-angle_f)), 6)
    y_y = round(move_r * math.sin(math.radians(-angle_f)), 6)
#     x_x = move_point_x
#     y_y = -move_point_y
        
    sample = np.zeros((img_exp_x, img_exp_y), dtype=np.uint16)
    for i in range(len(sample_x)):
        temp_x = int(sample_x[i])
        temp_y = int(sample_y[i])
        sample[temp_x][temp_y] = 1
    sample = rotate(sample, angle=-ref_angle, reshape=False)
    sample_temp_x, sample_temp_y = np.where(sample==1)
        
    h_p_x = round(h_p_x + x_x, 6)
    h_p_y = round(h_p_y + y_y, 6)
    
    lab2[-1].value="x, y, r: {} [']".format((x_x, y_y, move_r))
    lab3[-1].value="hx, hy, af: {} [']".format((h_p_x, h_p_y, angle_f))
#     print(angle_now, end=" ")
    ref = math.sqrt(math.pow(ref_point_x, 2) + math.pow(ref_point_y, 2))
    h_p = math.sqrt(math.pow(h_p_x, 2) + math.pow(h_p_y, 2))
    
    if math.fabs(h_p - ref) >= 7:
        ref_point_x = int(h_p_x)
        ref_point_y = int(h_p_y)
        ref = math.sqrt(math.pow(ref_point_x, 2) + math.pow(ref_point_y, 2))
        trace.append(tuple((ref_point_y, ref_point_x)))
        cv2.destroyAllWindows()
        photo_map(sample_temp_x, sample_temp_y, ref_point_x, ref_point_y, trace)
        move_point_x = 0
        move_point_y = 0
        print("!paint!", end=" ")

#